In [24]:
from model import TMapper, PhiMapper, OmegaMapper, UnbalancedLoss
import torch

In [30]:
from model import Hellinger, Jensen_Shannon, KL_dual, Pearson_xi

In [25]:
import torch.nn.functional as F

In [28]:
def cost_matrix(first:torch.Tensor, second:torch.Tensor):
    return torch.pairwise_distance(first.unsqueeze(2), second)

def mass_variation(s):
    return 0.01 * s

In [27]:
T = TMapper(10, 5)
Xi = PhiMapper(10, 1)
f = OmegaMapper(5, 1)
loss = UnbalancedLoss(10, 5)
w_optim = torch.optim.SGD(f.parameters(), lr=1e-2)
t_optim = torch.optim.SGD(T.parameters(), lr=1e-2)
xi_optim = torch.optim.SGD(Xi.parameters(), lr=1e-2)

In [31]:
def train_loop(data_loader: torch.utils.data.DataLoader):
    for X, Z, Y in data_loader:
        
        w_optim.zero_grad()
        t_optim.zero_grad()
        xi_optim.zero_grad()
        
        loss_value = -loss.compute(X, Z, Y, T(X, Z), Xi(X), f(Y), f(T(X, Z)))
        print('-loss value {}'.format(loss_value))
        loss_value.backward()
        w_optim.step()
        
        t_optim.zero_grad()
        xi_optim.zero_grad()
        loss_value = loss.compute(X, Z, Y, T(X, Z), Xi(X), f(Y), f(T(X, Z)))
        print('loss value {}'.format(loss_value))
        loss_value.backward()
        t_optim.step()
        xi_optim.step()